In [1]:
!pip install xlrd

In [2]:
import numpy as np
import pandas as pd
import xlrd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/KOPIS공모전/활용데이터 및 코드/할인율학습데이터/data_w_기존컬럼.csv')  # 데이터 파일 로드
# 데이터 프레임 가정
df.info()

<ipython-input-4-76ca63fd5163>:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/KOPIS공모전/활용데이터 및 코드/할인율학습데이터/data_w_기존컬럼.csv')  # 데이터 파일 로드


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743859 entries, 0 to 6743858
Data columns (total 42 columns):
 #   Column           Dtype  
---  ------           -----  
 0   좌석수              int64  
 1   공연코드             object 
 2   공연일시             object 
 3   예매/취소일시          object 
 4   예매/취소구분          int64  
 5   예매/취소매수          int64  
 6   예매/취소금액          float64
 7   예매/취소방식명(관리시스템)  object 
 8   할인금액             int64  
 9   할인종류명(전송처)       object 
 10  장당금액             int64  
 11  성별               int64  
 12  공연시작일자           object 
 13  공연종료일자           object 
 14  장르명              int64  
 15  출연진내용            object 
 16  아동공연 여부          object 
 17  내한공연 여부          object 
 18  오픈런 여부           object 
 19  판매시작일시           object 
 20  판매종료일시           object 
 21  좌석등급             object 
 22  극장규모             int64  
 23  재관람              int64  
 24  예매/취소방식명(전송처)    object 
 25  스타배우             int64  
 26  할인율              float64
 27  AGE         

In [5]:
df.shape

(6743859, 42)

In [6]:
df['공연횟수'] = df.groupby('공연코드')['공연일시'].transform('nunique')

In [7]:
df = df.drop(columns=['예매/취소매수', '예매/취소구분', '성별', '출연진내용', '좌석수', '장르명', '좌석등급',
                      '극장규모', '예매/취소방식명(전송처)', '할인종류명(전송처)'])

In [8]:
# 날짜 특성 생성
df['공연일시'] = pd.to_datetime(df['공연일시'])
df['공연일시_년'] = df['공연일시'].dt.year
df['공연일시_월'] = df['공연일시'].dt.month
df['공연일시_일'] = df['공연일시'].dt.day
df['공연일시_요일'] = df['공연일시'].dt.weekday
df = df.drop(columns=['공연일시'])

In [9]:
df.columns

Index(['공연코드', '예매/취소일시', '예매/취소금액', '예매/취소방식명(관리시스템)', '할인금액', '장당금액',
       '공연시작일자', '공연종료일자', '아동공연 여부', '내한공연 여부', '오픈런 여부', '판매시작일시', '판매종료일시',
       '재관람', '스타배우', '할인율', 'AGE', 'cluster', '공연지역명_경기도', '공연지역명_경상도',
       '공연지역명_서울', '공연지역명_전라도', '공연지역명_제주도', '공연지역명_충청도', '계층_조합_뮤지컬_1',
       '계층_조합_뮤지컬_2', '계층_조합_연극_0', '계층_조합_연극_1', '계층_조합_연극_2', '성별_1', '성별_2',
       '공연횟수', '공연일시_년', '공연일시_월', '공연일시_일', '공연일시_요일'],
      dtype='object')

In [10]:
from sklearn.preprocessing import LabelEncoder

# 이진 변수 변환 (0과 1로)
le = LabelEncoder()

# 이진 변수인 '장르명'과 '예매/취소방식명'을 숫자형으로 변환
df['아동공연 여부'] = le.fit_transform(df['아동공연 여부'])
df['내한공연 여부'] = le.fit_transform(df['내한공연 여부'])
df['오픈런 여부'] = le.fit_transform(df['오픈런 여부'])

In [11]:
df.head()

,공연코드,예매/취소일시,예매/취소금액,예매/취소방식명(관리시스템),할인금액,장당금액,공연시작일자,공연종료일자,아동공연 여부,내한공연 여부,...,계층_조합_연극_0,계층_조합_연극_1,계층_조합_연극_2,성별_1,성별_2,공연횟수,공연일시_년,공연일시_월,공연일시_일,공연일시_요일
0,PF304959,2022-05-02 09:38,69000.0,모바일,0,69000,2021-11-01,2023-07-30,0,0,...,True,False,False,False,True,1019,2022,6,2,3
1,PF300599,2022-06-08 22:22,26400.0,웹,0,26400,2022-05-24,2022-08-21,0,0,...,False,False,False,False,False,117,2022,6,9,3
2,PF308077,2022-05-24 09:56,135000.0,모바일,0,135000,2022-05-10,2022-08-07,0,0,...,False,False,False,False,True,102,2022,6,9,3
3,PF312815,2022-06-09 13:00,27000.0,모바일,0,27000,2022-05-12,2022-07-10,0,0,...,True,False,False,False,True,36,2022,6,9,3
4,PF308025,2022-05-11 19:04,46200.0,웹,19800,66000,2022-03-22,2022-06-19,0,0,...,False,False,False,False,True,84,2022,6,4,5


In [12]:
df.isnull().sum()

,0
공연코드,0
예매/취소일시,0
예매/취소금액,78682
예매/취소방식명(관리시스템),0
할인금액,0
장당금액,0
공연시작일자,0
공연종료일자,0
아동공연 여부,0
내한공연 여부,0


In [13]:
df = df.dropna()

In [14]:
# '공연일시'는 별도로 추출
tmp = df[['예매/취소일시', '공연시작일자', '공연종료일자', '판매시작일시', '판매종료일시', '공연코드']]
df = df.drop(columns=['예매/취소일시', '공연시작일자', '공연종료일자', '판매시작일시', '판매종료일시', '공연코드'])  # '공연코드'는 그룹핑에 사용되므로 제외

In [15]:
# 범주형 변수 인코딩
categorical_columns = ['예매/취소방식명(관리시스템)']
encoder = OneHotEncoder(sparse_output=False, drop='first')
X_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), index=df.index,  columns=encoder.get_feature_names_out(categorical_columns))
df = df.drop(categorical_columns, axis=1)
df = pd.concat([df, X_encoded], axis=1)

In [16]:
df.columns = df.columns.astype(str)

In [17]:
df.columns

Index(['예매/취소금액', '할인금액', '장당금액', '아동공연 여부', '내한공연 여부', '오픈런 여부', '재관람',
       '스타배우', '할인율', 'AGE', 'cluster', '공연지역명_경기도', '공연지역명_경상도', '공연지역명_서울',
       '공연지역명_전라도', '공연지역명_제주도', '공연지역명_충청도', '계층_조합_뮤지컬_1', '계층_조합_뮤지컬_2',
       '계층_조합_연극_0', '계층_조합_연극_1', '계층_조합_연극_2', '성별_1', '성별_2', '공연횟수',
       '공연일시_년', '공연일시_월', '공연일시_일', '공연일시_요일', '예매/취소방식명(관리시스템)_웹'],
      dtype='object')

In [18]:
# 필요한 컬럼 추출
features = ['예매/취소금액', '할인금액', '장당금액', '아동공연 여부', '내한공연 여부', '오픈런 여부', '재관람',
       '스타배우', 'AGE', 'cluster', '공연지역명_경기도', '공연지역명_경상도', '공연지역명_서울', '공연지역명_전라도',
       '공연지역명_제주도', '공연지역명_충청도', '계층_조합_뮤지컬_1', '계층_조합_뮤지컬_2', '계층_조합_연극_0',
       '계층_조합_연극_1', '계층_조합_연극_2', '성별_1', '성별_2', '공연횟수', '공연일시_년', '공연일시_월',
       '공연일시_일', '공연일시_요일', '예매/취소방식명(관리시스템)_웹']

target = '할인율'

In [19]:
# 스케일링
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df.drop(columns=[target]))

# 스케일링 후 데이터프레임으로 변환
X_scaled_df = pd.DataFrame(X_scaled, columns=df.drop(columns=[target]).columns)

In [20]:
# '공연코드' 열 추출
performance_codes = tmp[['공연코드']]

# 인덱스가 일치하는 경우 join으로 추가
df = df.join(performance_codes)

In [21]:
# 슬라이딩 윈도우 생성 함수
def create_sliding_windows(data, target, window_size):
    X_windows = []
    y_windows = []
    for i in range(len(data) - window_size +1):
        X_windows.append(data[i:i + window_size])
        y_windows.append(target[i + window_size -1])
    return np.array(X_windows), np.array(y_windows)

In [22]:
# 시계열 데이터 생성
# 클러스터 별 공연코드로 같은 공연 묶기, 이후 공연일시로 시계열 데이터 생성

# 시계열 데이터 생성
df.sort_values(by=['cluster', '공연코드', '공연일시_년', '공연일시_월', '공연일시_일'], inplace=True)


In [ ]:
df.head(15)

,예매/취소금액,할인금액,장당금액,아동공연 여부,내한공연 여부,오픈런 여부,재관람,스타배우,할인율,AGE,...,계층_조합_연극_2,성별_1,성별_2,공연횟수,공연일시_년,공연일시_월,공연일시_일,공연일시_요일,예매/취소방식명(관리시스템)_웹,공연코드
3693996,30800.0,0,30800,0,0,0,0,0,0.0,51.0,...,False,False,True,84,2022,4,26,1,0.0,PF300001
3714177,35750.0,0,35750,0,0,0,0,0,0.0,47.0,...,False,False,True,84,2022,4,26,1,0.0,PF300001
3717022,35750.0,0,35750,0,0,0,0,0,0.0,47.0,...,False,False,True,84,2022,4,26,1,0.0,PF300001
3717354,35750.0,0,35750,0,0,0,0,0,0.0,54.0,...,False,False,True,84,2022,4,26,1,0.0,PF300001
3719392,35750.0,0,35750,0,0,0,0,0,0.0,51.0,...,False,False,True,84,2022,4,26,1,1.0,PF300001
3687441,28600.0,0,28600,0,0,0,0,0,0.0,51.0,...,False,False,True,84,2022,4,27,2,1.0,PF300001
3687642,35750.0,0,35750,0,0,0,0,0,0.0,58.0,...,False,False,True,84,2022,4,27,2,1.0,PF300001
3699583,35750.0,0,35750,0,0,0,0,0,0.0,58.0,...,False,False,True,84,2022,4,27,2,1.0,PF300001
3706305,38500.0,0,38500,0,0,0,0,0,0.0,51.0,...,False,False,True,84,2022,4,27,2,1.0,PF300001
3710159,38500.0,0,38500,0,0,0,0,0,0.0,55.0,...,False,True,False,84,2022,4,27,2,1.0,PF300001


In [ ]:
# 클러스터별 데이터 분리 및 LSTM 모델 학습
# 수정 전 모델
cluster_models = {}
for cluster_id in df['cluster'].unique():
    cluster_data = df[df['cluster'] == cluster_id]

    # 공연 횟수에 따른 데이터 필터링
    # 단발성 공연 필터링
    cluster_data = cluster_data[cluster_data['공연횟수'] >= 3]

    # 공연 횟수에 따른 윈도우 크기 조정
    # 30회 이상인 공연의 슬라이딩 윈도우 크기는 30
    # 30회 미만인 공연의 슬라이딩 윈도우 크기는 3
    # 공연코드별로 그룹화하여 LSTM 학습
    for performance_code in cluster_data['공연코드'].unique():
        performance_data = cluster_data[cluster_data['공연코드'] == performance_code]

        # 공연코드별로 공연횟수 확인 후 윈도우 크기 조정
        show_count = performance_data['공연횟수'].iloc[0]  # 해당 공연의 공연횟수를 가져옴

        if show_count >= 30:
            window_size = 30
        else:
            window_size = 3

    # 스케일링
    X_scaled_cluster = scaler.transform(X_cluster)

    # 슬라이딩 윈도우 생성
    X_windows, y_windows = create_sliding_windows(X_scaled_cluster, y_cluster.values, window_size)

    # 데이터셋 분리
    X_train, X_test, y_train, y_test = train_test_split(X_windows, y_windows, test_size=0.2, random_state=42)

    # LSTM 모델 구성
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(window_size, X_train.shape[2])))
    model.add(LSTM(units=50))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')

    # 모델 학습
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    # 모델 저장
    cluster_models[cluster_id] = model

KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint, LambdaCallback

# 클러스터별 LSTM 모델 학습
cluster_models = {}
scaler = MinMaxScaler()

for cluster_id in df['cluster'].unique():
    cluster_data = df[df['cluster'] == cluster_id]

    # 공연 횟수에 따른 데이터 필터링
    cluster_data = cluster_data[cluster_data['공연횟수'] >= 3]

    # 공연코드별로 그룹화하여 LSTM 학습
    for performance_code in cluster_data['공연코드'].unique():
        performance_data = cluster_data[cluster_data['공연코드'] == performance_code]

        # 공연코드별로 공연횟수 확인 후 윈도우 크기 조정
        show_count = performance_data['공연횟수'].iloc[0]  # 해당 공연의 공연횟수를 가져옴

        if show_count >= 30:
            window_size = 30
        else:
            window_size = 3

        # 스케일링
        X_cluster = performance_data[features]
        y_cluster = performance_data[target]
        X_scaled_cluster = scaler.fit_transform(X_cluster)

        # 슬라이딩 윈도우 생성
        X_windows, y_windows = create_sliding_windows(
            X_scaled_cluster,
            y_cluster.values,
            window_size
        )

        # 데이터셋 분리
        if len(X_windows) > 1:  # 데이터가 2개 이상일 때만 train_test_split 사용
            X_train, X_test, y_train, y_test = train_test_split(X_windows, y_windows, test_size=0.2, random_state=42)
        else:
            X_train, y_train = X_windows, y_windows
            X_test, y_test = [], []

        # 리스트를 numpy 배열로 변환
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)

        # 데이터 타입 변환
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        y_train = y_train.astype('float32')
        y_test = y_test.astype('float32')

        # LSTM 모델 구성 및 학습
        if len(X_train) > 0:
            model = Sequential()
            model.add(Input(shape=(window_size, X_train.shape[2])))
            model.add(LSTM(units=15, return_sequences=True))
            model.add(LSTM(units=15))
            model.add(Dense(1))

            model.compile(optimizer='adam', loss='mse')

           # 체크포인트 콜백: 에포크가 끝날 때마다 모델 저장
            checkpoint = ModelCheckpoint(f'model_cluster_{cluster_id}_performance_{performance_code}.keras', save_best_only=True)

            # 진행 상황을 출력하는 콜백
            def print_progress(epoch, logs):
                progress = (epoch + 1) / 10  # 전체 에포크 대비 현재 에포크 비율 (여기선 10에포크가 고정이므로 10으로 나눔)
                print(f"Cluster {cluster_id}, Performance {performance_code}: {progress*100:.2f}% complete.")

            progress_callback = LambdaCallback(on_epoch_end=print_progress)

            if len(X_test) > 0:  # 테스트 데이터가 있을 때만 validation_data 사용
                model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
            else:  # 테스트 데이터가 없으면 그냥 훈련만 진행
                model.fit(X_train, y_train, epochs=10, batch_size=32)

            # 모델 저장
            cluster_models[(cluster_id, performance_code)] = model


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - loss: 0.0387 - val_loss: 0.0310
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0290 - val_loss: 0.0261
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0262 - val_loss: 0.0214
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0254 - val_loss: 0.0174
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0190 - val_loss: 0.0149
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0163 - val_loss: 0.0124
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0160 - val_loss: 0.0113
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0161 - val_loss: 0.0113
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0152 - val_loss: 0.0111
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0168 - val_loss: 0.0111
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - loss: 0.0305 - val_loss: 0.0070
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.

In [ ]:
# NaN이나 Inf가 있는지 확인
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))
print(np.any(np.isnan(y_train)))
print(np.any(np.isinf(y_train)))

False
False
False
False


In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (94, 3, 29)
y_train shape: (94,)
X_test shape: (24, 3, 29)
y_test shape: (24,)


In [ ]:
print(f"X_train dtype: {X_train.dtype}")
print(f"y_train dtype: {y_train.dtype}")

X_train dtype: float64
y_train dtype: float64


할인율을 예측하는 코드를 제작하였다.